<a href="https://colab.research.google.com/github/KevinTheRainmaker/Recommendation_Algorithms/blob/main/colab/tf_recommender/01_TensorFlow_Recommender_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# Recommending Movies

Real-world의 Recommender는 두 단계로 이루어지곤 한다.
1. The Retrieval Stage:

  전체 후보군으로부터 초기 후보군을 골라내는 단계. 유저가 관심을 가지지 않을 후보를 제거함으로써 계산에 효율성을 더한다.

2. The Ranking Stage:

  Retrieval model의 결과를 인풋으로 받아 fine-tuning을 통해 최적의 recommendation을 선별하는 단계. Item의 set을 narrow down 시켜 최적 후보군 리스트를 만들어낸다.


## The Retrieval Model

Retrieval Model은 두 개의 sub-model로 이루어진다.

1. Query feature를 이용해서 Query representation을 계산하는 Query Model
2. Candidate feature를 이용해서 Candidate representation을 계산하는 Candidate Model

두 모델의 output은 서로 multiple되어 query-candidate affinity score를 출력한다.

### Process

1. 데이터를 훈련세트와 검증세트로 나눈다.
2. Retrieval Model을 Implement
3. Fit & Evaluate
4. 더 효율적인 서빙을 위해 Approximate Nearest Neighbours (ANN) index를 Build하여 Export

### Import Packages

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

ScaNN (Scalable Nearest Neighbors): 구글이 개발한 ANN 알고리즘 중 하나로, 거의 모든 Queries per seconds에서 가장 최적의 Recall 값을 낸 알고리즘이다.

In [ ]:
import os
import pprint
import tempfile

import numpy as np

from typing import Dict, Text

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Dataset

데이터는 유명한 영화 데이터인 movielens를 사용한다.

이 데이터는 다음과 같이 두 가지 방식으로 다뤄질 수 있다.
1. Implicit Feedback: 유저가 어떤 영화를 보았는지
2. Explicit Feedback: 유저가 본 영화에 대해서 어떤 영화를 얼마나 선호하는지

Retireval Model을 다루는 파트에서는 Implicit system에 집중하여, 유저가 본 영화는 Positive example로, 보지 않은 영화는 Implicit negative example로 간주하여 진행하도록 하겠다.

\* Implicit Negative Example이란: 보지 않은 영화라고해서 전부 유저가 선호하지 않는 것은 아니다. 이는 Implicit Feedback에서 주로 사용되는 개념으로, Implicit Negative는 실제로 유저가 선호하지 않는 Real Negative와 유저가 미래에 소비할지도 모르는 아이템이지만 아직 소비하지 않은, Missing Value가 혼합되어 있다. 

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [ ]:
print(len(ratings), len(movies))

100000 1682


In [ ]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


장르가 integer labels로 encodding되어있음을 알 수 있다.

In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

`user_id`와 `movie_title`만을 남기도록 하자.

Fit과 Evaluate를 수행하기 위해서는 데이터셋을 훈련세트와 검증세트로 나누어야 한다.

실무 환경에서 사용될 Industrial Model에서는 이것이 시간에 기반하여 이루어진다. 즉, 시간 T까지 수집된 데이터는 T 이후의 예측을 수행하기 위해 사용된다.

여기서는 이미 수집된 데이터를 이용하는 것이므로, random split 방식을 사용하도록 하겠다.

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
# figure out unique user ids and movie titles
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [ ]:
unique_user_ids[:10]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)

### Implementing a Model

모델 아키텍처를 선택하는 것은 전체 모델링의 핵심부이다.

여기서 우리는 Two-tower Retrieval Model을 구축할 것이고, 각각을 개별적으로 구축한 후 final model로 통합시킬 것이다.

#### The Query Tower

In [ ]:
# decide the dimensionality of the query and candidate representations
embedding_dimension = 32

값을 높게 잡으면 더욱 정확한 모델이 될 수 있지만, 동시에 학습속도가 느려지고 overfitting의 우려 또한 커진다.

In [ ]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

#### The Candidate Tower

In [ ]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

#### Metrics

우리의 학습 데이터에는 Positive (유저, 영화)쌍이 포함되어 있다. 여기서 Positive하다는 것은, 실제로 유저가 본 영화가 서로 묶인 쌍이라는 뜻이다.

이러한 쌍이 다른 모든 가능한 (유저, 영화)쌍보다 높은 affinity score를 가진다면, 우리의 모델은 정확하다고 할 수 있다.

이를 위해 우리는 `tfrs.metrics.FactorizedTopK` metric을 사용할 것이다. 이는 implicit negative로 사용된 후보군 데이터를 인자로 가진다.

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

#### Loss

다음으로, 우리의 모델을 학습시킬 때 필요한 Lossㄹ르 정의해보자.

TFRS는 이를 편리하게 해주는 몇 가지 Loss layer와 tasks를 가지고있다.

여기서 우리는 `Retrieval`이라는 task object를 사용할 것이다.

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

이 task object는 그 자체로 Keras layer로 동작하며, query embedding과 candidate embedding을 인자로 받아서 loss를 반환한다.

#### The Full model

TFRS는 Base model class인 `tfrs.models.Model`을 제공한다. 이 덕분에 `__init__` 메서드 안에 components를 set up하고 `compute_loss` 메서드를 implement만 하면 된다.

In [ ]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

`tfrs.Model` base class는 training loss와 test loss를 같은 메서드를 사용해서 계산할 수 있도록 해주는 편리한 class이다.

이 외에 `tf.keras.Model`로부터 inherit 받아 `train_step`과 `test_step`을 오버라이딩 시켜 같은 기능을 구현할 수도 있다.

In [ ]:
class NoBaseClassMovielensModel(tf.keras.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.user_model(features["user_id"])
      positive_movie_embeddings = self.movie_model(features["movie_title"])
      loss = self.task(user_embeddings, positive_movie_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.user_model(features["user_id"])
    positive_movie_embeddings = self.movie_model(features["movie_title"])
    loss = self.task(user_embeddings, positive_movie_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

여기서는 모델링에 집중하고 보일러 플레이트(boilerplate)로부터 벗어나기 위해 `tfrs.Model`을 사용하겠다.

### Fit & Evaluate

In [ ]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
# shuffle, batch, and cache the training & evaluation data
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 15s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0013 - factorized_top_k/top_10_categorical_accuracy: 0.0040 - factorized_top_k/top_50_categorical_accuracy: 0.0443 - factorized_top_k/top_100_categorical_accuracy: 0.1070 - loss: 69885.1094 - regularization_loss: 0.0000e+00 - total_loss: 69885.1094
Epoch 2/3
10/10 [==============================] - 10s 1s/step - factorized_top_k/top_1_categorical_accuracy: 9.3750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0108 - factorized_top_k/top_10_categorical_accuracy: 0.0250 - factorized_top_k/top_50_categorical_accuracy: 0.1404 - factorized_top_k/top_100_categorical_accuracy: 0.2633 - loss: 67523.3693 - regularization_loss: 0.0000e+00 - total_loss: 67523.3693
Epoch 3/3
5/5 [==============================] - 6s 556ms/step - factorized_top_k/top_1_categorical_accuracy: 9.0000e-04 - factorized_top_k/top_5_categorical_accu

{'factorized_top_k/top_100_categorical_accuracy': 0.23280000686645508,
 'factorized_top_k/top_10_categorical_accuracy': 0.022299999371170998,
 'factorized_top_k/top_1_categorical_accuracy': 0.0008999999845400453,
 'factorized_top_k/top_50_categorical_accuracy': 0.12470000237226486,
 'factorized_top_k/top_5_categorical_accuracy': 0.009399999864399433,
 'loss': 28244.7734375,
 'regularization_loss': 0,
 'total_loss': 28244.7734375}

test set에 대한 performance가 training set에 대해서보다 훨씬 안 좋다. 이러한 현상의 이유로는 두 가지 정도를 생각할 수 있다.

1. 너무 많은 feature 등의 이유로 모델이 overfitting 되었다. 이는 regularization 혹은 feature engineering을 통해 새로운 데이터에 대한 예측을 돕는 feature를 추가하는 식으로 조절할 수 있다.

2. 모델이 items_known을 추천하고 있다. Top-K에 대해서 정확도를 계산하기 때문에 중복 추천이 발생할 경우 test set에서의 performance가 안 좋을 수 있다. 이 경우는 재추천을 허용하지 않음으로써 막을 수 있다.

### Making Predictions

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

In [ ]:
# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)' b'Rudy (1993)']


`BruteForce` layer는 많은 후보군을 가지는 경우 모델 서빙을 느리게 만든다. 따라서, 적절한 retrieval index를 이용해 이를 빠르게 만드는 방법에 대해 알아보자.

### Model Serving

모델 학습이 완료된 후, 우리는 이것을 배포하는 방법이 필요하다.

위와 같은 Two-tower Retrieval model의 경우, 서빙에는 다음과 같은 두 개의 components가 포함된다.

1. query의 feature를 받아 query embedding으로 만드는 query model의 서빙
2. query model의 결과로 나온 query에 대하여 후보군을 빠르게 찾는 ANN 모델의 인덱스를 받아 후보군을 생성하는 candidate model의 서빙

In [ ]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["42"])

  print(f"Recommendations: {titles[0][:3]}")

INFO:tensorflow:Assets written to: /tmp/tmpzcqbd36r/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpzcqbd36r/model/assets


Recommendations: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)' b'Rudy (1993)']


또한 우리는 예측의 속도를 높이기 위해 적절한 retrieval index를 export할 수도 있는데, 이는 수많은 후보군에 대해 빠르게 예측을 수행할 수 있도록 한다.

In [ ]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.user_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

In [ ]:
# Get recommendations.
_, titles = scann_index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Sleepless in Seattle (1993)' b'Father of the Bride Part II (1995)'
 b'Hunchback of Notre Dame, The (1996)']


In [ ]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(
      index,
      path,
      options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
  )

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["42"])

  print(f"Recommendations: {titles[0][:3]}")

INFO:tensorflow:Assets written to: /tmp/tmp9tcyzpad/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp9tcyzpad/model/assets


Recommendations: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)' b'Rudy (1993)']


To learn more about using and tuning fast approximate retrieval models, have a look at [efficient serving](https://tensorflow.org/recommenders/examples/efficient_serving) tutorial.

### +) Item-to-Item recommendation

여기서, 우리는 user-movie model을 만들었다. 하지만, 일부 상황에서는 item-item model이 필요한 경우 또한 존재한다.

이러한 모델은 전반적인 과정이 위와 동일하나 다른 데이터를 사용해야한다. 여기서는 user와 movie tower를 통해 (유저, 영화) 쌍을 학습에 이용했지만 item-item model에서는 두 개의 item tower를 이용해야 할 것이다. 

데이터 예시: 물품 상세보기 페이지 클릭

##The Ranking Model

위에서 구현한 Retrieval Model의 결과값으로 나온 후보군에 대하여 랭킹을 계산하고 이를 통해 최종 추천 항목을 구하는 모델이다.

### Import Packages


위에서 진행한 부분이지만, 추후 이 부분만을 다시 돌릴 수 있도록 처음부터 진행하겠다.

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Preparing Dataset

In [ ]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

Retrieval에서와 같은 데이터를 사용한다. 차이점은, Retrieval 모델에서 단순히 유저와 영화만의 관계를 이용했다면, 여기서는 해당 유저가 영화에 준 rating 값 또한 학습에 이용하도록 하겠다. 

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

### Implementing a Model

### Architecture

Ranking Model은 Retrieval Model과 달리 efficiency constraints를 지킬 필요가 없다.

모델은 다중 stacked dense layer로 이루어져있고, 이는 Ranking에 사용되는 공통된 아키텍쳐를 가진다.

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

`RankingModel`은 유저 아이디와 영화 제목을 받아 예측 평점을 출력한다.

In [ ]:
RankingModel()((["42"], ["Bridges of Madison County, The (1995)"]))

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.0076181]], dtype=float32)>

### Loss and Metrics

TFRS에서 Ranking Task를 위해 제공하는 Loss와 Metric object를 사용하겠다.

In [ ]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

위 태스크는 그 자체로 하나의 Keras layer이고, 실제 값과 예측 값은 인자로 받아 loss를 출력한다.

### The Full Model

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

### Fitting and Evaluating

In [ ]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 4s 28ms/step - root_mean_squared_error: 2.0840 - loss: 3.9935 - regularization_loss: 0.0000e+00 - total_loss: 3.9935
Epoch 2/3
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 1.1253 - loss: 1.2617 - regularization_loss: 0.0000e+00 - total_loss: 1.2617
Epoch 3/3
10/10 [==============================] - 0s 12ms/step - root_mean_squared_error: 1.1178 - loss: 1.2437 - regularization_loss: 0.0000e+00 - total_loss: 1.2437


학습이 진행될 수록 loss가 떨어지는 것을 확인할 수 있다.

마지막으로 test set에 대해 evaluate 해보자.

In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 15ms/step - root_mean_squared_error: 1.1086 - loss: 1.2241 - regularization_loss: 0.0000e+00 - total_loss: 1.2241


{'loss': 1.2020609378814697,
 'regularization_loss': 0,
 'root_mean_squared_error': 1.1086448431015015,
 'total_loss': 1.2020609378814697}

### Testing the Ranking Model

In [ ]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
Dances with Wolves (1990): [[3.6997178]]
M*A*S*H (1970): [[3.6699927]]
Speed (1994): [[3.6349838]]


### Exporting for Serving

모델은 serving을 통해 쉽게 export될 수 있다.

In [ ]:
tf.saved_model.save(model, "export")

INFO:tensorflow:Assets written to: export/assets


INFO:tensorflow:Assets written to: export/assets


In [ ]:
loaded = tf.saved_model.load("export")

loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()

array([[3.6349838]], dtype=float32)

## Wrap-up


이로써 기본적인 Retrieval Model과 Ranking Model의 구축이 끝났다.

물론 이것은 굉장히 간단한 모델이고, 실무에서 사용되는 모델은 이보다 훨씬 복잡할 것이다.
대부분의 경우에 Ranking Model은 여기서와 달리 유저-아이템의 identifier 외에도 많은 feature를 사용하여 Ranking을 계산한다. 이에 대해서는 TensorFlow_Recommender_Using_Rich_Features에서 다룰 것이다.

또한 objective worth optimizing에 대한 이해가 필요한데, 이는 TensorFlow_Recommender_Multitask에서 다루도록 하겠다.